In [1]:
import pandas as pd
import numpy as np
import os
import json

# Classifier Results

The directory structure for the Classifier is: We have the classifier directory (in the path variable), then in this directory we have one sub directory for each dataset (these subdirectories are given to the ICR as a input flag). In the directory for the dataset we have one directory (created by ICR) for each confidence level per classifier being used.

In [5]:
path = "../results/classifiers/"

In [6]:
df = pd.DataFrame()

In [7]:
for dataset in os.listdir(path):
    for run in os.listdir(path+dataset): 
        if os.path.isdir(path+dataset+"/"+run):
            try:
                df_curr = pd.read_csv(path+dataset+"/"+run+'/aggregated_result.csv')
                with open(path+dataset+"/"+run+'/problem_setup.json') as f:
                    setup = json.load(f)
                df_curr['dataset'] = setup['scm_name']
                df_curr['model_type'] = setup['model_type']
                df_curr['confidence'] = setup['gamma/eta']
                df = pd.concat([df, df_curr], ignore_index=True)
            except:
                pass

In [8]:
df['method'] = df['r_type']+" "+df['t_type']

Filtering only confidence level 95% for the plotting, however 75 gives similar results

In [9]:
df = df[df.confidence == 0.95]

In [10]:
pivot_df_eta = df.pivot_table(values='eta_obs_mean', index=['method'], columns=['model_type'], 
                              aggfunc=['mean', 'std'])

In [11]:
pivot_df_refit = df.pivot_table(values='eta_obs_refits_batch0_mean_mean', index=['method'], 
                                columns=['model_type'], aggfunc=['mean', 'std'])

In [12]:
pivot_df_gamma = df.pivot_table(values='gamma_obs_mean', index=['method'], columns=['model_type'], 
                                aggfunc=['mean', 'std'])

In [13]:
pivot_df_cost = df.pivot_table(values='intv-cost_mean', index=['method'], columns=['model_type'], 
                               aggfunc=['mean', 'std'])

In [14]:
def create_classifier_pivot(df):
    classifiers = ['MLP', 'SVC', 'adaboost', 'logreg', 'rf']
    pivot_df = df.copy()
    pivot_df.columns = [f'{agg}_{shift}' for agg, shift in pivot_df.columns]
    
    pivot_df = pivot_df.round(2)
    
    old_columns = list(pivot_df.columns)
    
    for i in classifiers:
        pivot_df[i] = pivot_df.apply(lambda row: f"${row['mean_'+i]} \pm {row['std_'+i]}$", axis=1)
    
    return pivot_df.drop(columns=old_columns)

In [15]:
refit_table=create_classifier_pivot(pivot_df_refit)

In [16]:
refit_table

,MLP,SVC,adaboost,logreg,rf
method,,,,,
individualized acceptance,$0.9 \pm 0.05$,$0.78 \pm 0.1$,$0.99 \pm 0.0$,$0.89 \pm 0.15$,$0.98 \pm 0.0$
individualized counterfactual,$0.87 \pm 0.06$,$0.75 \pm 0.12$,$0.99 \pm 0.0$,$0.88 \pm 0.17$,$0.98 \pm 0.0$
individualized improvement,$1.0 \pm 0.0$,$1.0 \pm 0.0$,$0.9 \pm 0.02$,$1.0 \pm 0.0$,$0.99 \pm 0.01$
subpopulation acceptance,$0.89 \pm 0.05$,$0.79 \pm 0.1$,$0.99 \pm 0.0$,$0.89 \pm 0.14$,$0.98 \pm 0.0$
subpopulation improvement,$0.87 \pm 0.08$,$0.86 \pm 0.09$,$0.93 \pm 0.02$,$0.86 \pm 0.1$,$0.85 \pm 0.1$


In [17]:
def print_latex(df):
    print(df.to_latex().replace('\\textbackslash pm', '\pm').replace('\\$', '$'))

In [18]:
print_latex(refit_table)

\begin{tabular}{llllll}
\toprule
{} &              MLP &              SVC &         adaboost &           logreg &               rf \\
method                        &                  &                  &                  &                  &                  \\
\midrule
individualized acceptance     &   $0.9 \pm 0.05$ &   $0.78 \pm 0.1$ &   $0.99 \pm 0.0$ &  $0.89 \pm 0.15$ &   $0.98 \pm 0.0$ \\
individualized counterfactual &  $0.87 \pm 0.06$ &  $0.75 \pm 0.12$ &   $0.99 \pm 0.0$ &  $0.88 \pm 0.17$ &   $0.98 \pm 0.0$ \\
individualized improvement    &    $1.0 \pm 0.0$ &    $1.0 \pm 0.0$ &   $0.9 \pm 0.02$ &    $1.0 \pm 0.0$ &  $0.99 \pm 0.01$ \\
subpopulation acceptance      &  $0.89 \pm 0.05$ &   $0.79 \pm 0.1$ &   $0.99 \pm 0.0$ &  $0.89 \pm 0.14$ &   $0.98 \pm 0.0$ \\
subpopulation improvement     &  $0.87 \pm 0.08$ &  $0.86 \pm 0.09$ &  $0.93 \pm 0.02$ &   $0.86 \pm 0.1$ &   $0.85 \pm 0.1$ \\
\bottomrule
\end{tabular}



In [19]:
gamma_table=create_classifier_pivot(pivot_df_gamma)

In [20]:
gamma_table

,MLP,SVC,adaboost,logreg,rf
method,,,,,
individualized acceptance,$0.61 \pm 0.56$,$0.58 \pm 0.6$,$0.3 \pm 0.25$,$0.58 \pm 0.59$,$0.77 \pm 0.3$
individualized counterfactual,$0.55 \pm 0.63$,$0.53 \pm 0.66$,$0.3 \pm 0.25$,$0.54 \pm 0.65$,$0.73 \pm 0.35$
individualized improvement,$1.0 \pm 0.0$,$1.0 \pm 0.0$,$1.0 \pm 0.0$,$1.0 \pm 0.0$,$1.0 \pm 0.0$
subpopulation acceptance,$0.6 \pm 0.57$,$0.56 \pm 0.62$,$0.3 \pm 0.25$,$0.57 \pm 0.61$,$0.76 \pm 0.31$
subpopulation improvement,$1.0 \pm 0.0$,$1.0 \pm 0.0$,$1.0 \pm 0.0$,$1.0 \pm 0.0$,$1.0 \pm 0.0$


# Shift Robustness

The directory structure for the Shifted Robustness is: We have the shift directory (in the path variable), then in this directory we have one sub directory for each dataset (these subdirectories are given to the ICR as a input flag). In the directory for the dataset we have one directory (created by ICR) for each confidence level per classifier being used. The results for the robustness can be found in each iteration directory in the robustness directory.

In [21]:
path = "../results/shifted_data/"

In [22]:
df = pd.DataFrame()

In [23]:
for dataset in os.listdir(path):
    if os.path.isdir(path+dataset):
        data_path = path+dataset+"/"
        for run in os.listdir(data_path):
            if os.path.isdir(data_path+run):
                with open(data_path+run+'/problem_setup.json') as f:
                    setup = json.load(f)
                for run_file in os.listdir(data_path+run): 
                    if os.path.isdir(data_path+run+"/"+run_file) and run_file != 'logs':
                        new_path = data_path+run+"/"+run_file+'/robustness/'
                        for shifted_run in os.listdir(new_path):
                            if os.path.isfile(new_path+shifted_run+'/stats_shift.json'):
                                model_type, t_type, r_type, variable, mean, var = shifted_run.split('-')
                                with open(new_path+shifted_run+'/stats_shift.json') as f:
                                    stats = json.load(f)
                                data_dict = {'model_type': model_type, 'recourse':t_type+"_"+r_type, 
                                             'variable': variable, 'mean':mean, 'var': var, 
                                             'dataset':dataset, 'confidence':setup['gamma/eta']}
                                data_dict.update(stats)
                                df = df.append(data_dict, ignore_index=True)

In [24]:
df['shift'] = df['mean']+'_'+df['var']

df = df[df['shift'] != 'mean0.0_var1.0']

df = df[df.confidence == 0.95]

shift_list = df['shift'].unique()

pivot_df = df.pivot_table(values='inv', index=['recourse', 'model_type'], columns=['shift'], 
                          aggfunc=['mean', 'std'])

pivot_df.columns = [f'{agg}_{shift}' for agg, shift in pivot_df.columns]

pivot_df = pivot_df.round(2)

old_columns = list(pivot_df.columns)

for i in shift_list:
    pivot_df['shift_'+i] = pivot_df.apply(lambda row: f"${row['mean_'+i]} \pm {row['std_'+i]}$", axis=1)

final_shift_df = pivot_df.drop(columns=old_columns)

In [25]:
final_shift_df

shift_mean0.0_var0.5  \
recourse                      model_type                        
acceptance_individualized     MLP             $0.51 \pm 0.33$   
                              SVC             $0.39 \pm 0.32$   
                              adaboost        $0.48 \pm 0.35$   
                              logreg          $0.45 \pm 0.37$   
                              rf              $0.44 \pm 0.35$   
acceptance_subpopulation      MLP             $0.49 \pm 0.34$   
                              SVC             $0.37 \pm 0.29$   
                              adaboost        $0.49 \pm 0.34$   
                              logreg          $0.44 \pm 0.35$   
                              rf              $0.47 \pm 0.36$   
counterfactual_individualized MLP             $0.56 \pm 0.39$   
                              SVC              $0.4 \pm 0.36$   
                              adaboost         $0.5 \pm 0.35$   
                              logreg          $0.48 \pm 0.38$   
                              rf              $0.47 \pm 0.37$   
improvement_individualized    MLP              $0.13 \pm 0.3$   
                              SVC              $0.13 \pm 0.3$   
                              adaboost        $0.15 \pm 0.28$   
                              logreg          $0.14 \pm 0.31$   
                              rf              $0.13 \pm 0.29$   
improvement_subpopulation     MLP             $0.14 \pm 0.31$   
                              SVC             $0.14 \pm 0.31$   
                              adaboost        $0.16 \pm 0.31$   
                              logreg          $0.15 \pm 0.32$   
                              rf              $0.14 \pm 0.31$   

                                         shift_mean0.5_var0.5  \
recourse                      model_type                        
acceptance_individualized     MLP             $0.79 \pm 0.37$   
                              SVC             $0.79 \pm 0.37$   
                              adaboost        $0.52 \pm 0.42$   
                              logreg          $0.79 \pm 0.38$   
                              rf              $0.67 \pm 0.41$   
acceptance_subpopulation      MLP             $0.79 \pm 0.36$   
                              SVC             $0.79 \pm 0.36$   
                              adaboost        $0.51 \pm 0.41$   
                              logreg          $0.78 \pm 0.38$   
                              rf              $0.64 \pm 0.41$   
counterfactual_individualized MLP             $0.83 \pm 0.38$   
                              SVC             $0.82 \pm 0.38$   
                              adaboost        $0.53 \pm 0.43$   
                              logreg          $0.81 \pm 0.38$   
                              rf              $0.67 \pm 0.42$   
improvement_individualized    MLP             $0.17 \pm 0.38$   
                              SVC             $0.17 \pm 0.38$   
                              adaboost        $0.21 \pm 0.36$   
                              logreg          $0.17 \pm 0.38$   
                              rf              $0.18 \pm 0.38$   
improvement_subpopulation     MLP              $0.2 \pm 0.35$   
                              SVC             $0.19 \pm 0.33$   
                              adaboost        $0.23 \pm 0.36$   
                              logreg           $0.2 \pm 0.34$   
                              rf              $0.23 \pm 0.37$   

                                         shift_mean0.5_var1.0  
recourse                      model_type                       
acceptance_individualized     MLP             $0.96 \pm 0.06$  
                              SVC             $0.94 \pm 0.08$  
                              adaboost        $0.66 \pm 0.38$  
                              logreg          $0.94 \pm 0.16$  
                              rf               $0.82 \pm 0.3$  
acceptance_subpopulation      MLP             $0.97 \pm 0.03$  
                 

In [26]:
print_latex(final_shift_df)

\begin{tabular}{lllll}
\toprule
                          &    & shift\_mean0.0\_var0.5 & shift\_mean0.5\_var0.5 & shift\_mean0.5\_var1.0 \\
recourse & model\_type &                      &                      &                      \\
\midrule
acceptance\_individualized & MLP &      $0.51 \pm 0.33$ &      $0.79 \pm 0.37$ &      $0.96 \pm 0.06$ \\
                          & SVC &      $0.39 \pm 0.32$ &      $0.79 \pm 0.37$ &      $0.94 \pm 0.08$ \\
                          & adaboost &      $0.48 \pm 0.35$ &      $0.52 \pm 0.42$ &      $0.66 \pm 0.38$ \\
                          & logreg &      $0.45 \pm 0.37$ &      $0.79 \pm 0.38$ &      $0.94 \pm 0.16$ \\
                          & rf &      $0.44 \pm 0.35$ &      $0.67 \pm 0.41$ &       $0.82 \pm 0.3$ \\
acceptance\_subpopulation & MLP &      $0.49 \pm 0.34$ &      $0.79 \pm 0.36$ &      $0.97 \pm 0.03$ \\
                          & SVC &      $0.37 \pm 0.29$ &      $0.79 \pm 0.36$ &      $0.95 \pm 0.06$ \\
                   

# Genetic algorithm

The directory structure for the Genetic Algorithm is: We have the main directory (in the path variable), then in this directory we have one sub directory for each genetic algorithm (these subdirectories are given to the ICR as a input flag). In the directory for the genetic algorithm we have one directory (created by ICR) for each confidence level per dataset being used.

In [27]:
path = "../results/genetic_algorithm/"

In [28]:
df = pd.DataFrame()

In [29]:
for genetic in os.listdir(path):
    for dataset in os.listdir(path+genetic):
        if os.path.isdir(path+genetic+"/"+dataset):
            with open(path+genetic+"/"+dataset+'/problem_setup.json') as f:
                setup = json.load(f)
            for iter in os.listdir(path+genetic+"/"+dataset):
                if os.path.isdir(path+genetic+"/"+dataset+"/"+iter) and iter != 'logs':
                    for method in os.listdir(path+genetic+"/"+dataset+"/"+iter):
                        if os.path.isdir(path+genetic+"/"+dataset+"/"+iter+"/"+method):
                            with open(path+genetic+"/"+dataset+"/"+iter+"/"+method+"/stats.json") as f:
                                stats = json.load(f)
                            data_dict = {'eta_refit':stats['eta_obs_refits_batch0_mean'],'eta':stats['eta_obs'],'gamma': stats['gamma_obs'], 'confidence': stats['gamma'], 
                                    'dataset': setup['scm_name'], 'method': method, 'genetic': setup['genetic_algo']}
                            df = df.append(data_dict, ignore_index=True)

In [30]:
df = df[df['confidence'] == 0.95]

In [31]:
df.groupby(['method', 'genetic'])['gamma'].mean()

method                         genetic
acceptance-individualized      nsga2      0.580000
                               nsga3      0.575000
acceptance-subpopulation       nsga2      0.566667
                               nsga3      0.568333
counterfactual-individualized  nsga2      0.536667
                               nsga3      0.540000
improvement-individualized     nsga2      1.000000
                               nsga3      1.000000
improvement-subpopulation      nsga2      0.998333
                               nsga3      1.000000
Name: gamma, dtype: float64

In [32]:
df.groupby(['method', 'genetic'])['gamma'].std()

method                         genetic
acceptance-individualized      nsga2      0.456640
                               nsga3      0.458814
acceptance-subpopulation       nsga2      0.471282
                               nsga3      0.469911
counterfactual-individualized  nsga2      0.503931
                               nsga3      0.500440
improvement-individualized     nsga2      0.000000
                               nsga3      0.000000
improvement-subpopulation      nsga2      0.004082
                               nsga3      0.000000
Name: gamma, dtype: float64

In [33]:
def create_genetic_pivot(df):
    genetic = ['nsga2', 'nsga3']
    pivot_df = df.copy()
    pivot_df.columns = [f'{agg}_{shift}' for agg, shift in pivot_df.columns]
    
    pivot_df = pivot_df.round(3)
    
    old_columns = list(pivot_df.columns)
    
    for i in genetic:
        pivot_df[i] = pivot_df.apply(lambda row: f"${row['mean_'+i]} \pm {row['std_'+i]}$", axis=1)
    
    return pivot_df.drop(columns=old_columns)

In [34]:
refit_pivot_genetic = df.pivot_table(values='eta_refit', index=['method'], columns=['genetic'], 
                                aggfunc=['mean', 'std'])

In [35]:
eta_pivot_genetic = df.pivot_table(values='eta', index=['method'], columns=['genetic'], 
                                aggfunc=['mean', 'std'])

In [36]:
gamma_pivot_genetic = df.pivot_table(values='gamma', index=['method'], columns=['genetic'], 
                                aggfunc=['mean', 'std'])

In [37]:
create_genetic_pivot(gamma_pivot_genetic)

,nsga2,nsga3
method,,
acceptance-individualized,$0.58 \pm 0.457$,$0.575 \pm 0.459$
acceptance-subpopulation,$0.567 \pm 0.471$,$0.568 \pm 0.47$
counterfactual-individualized,$0.537 \pm 0.504$,$0.54 \pm 0.5$
improvement-individualized,$1.0 \pm 0.0$,$1.0 \pm 0.0$
improvement-subpopulation,$0.998 \pm 0.004$,$1.0 \pm 0.0$


In [38]:
create_genetic_pivot(eta_pivot_genetic)

,nsga2,nsga3
method,,
acceptance-individualized,$1.0 \pm 0.0$,$1.0 \pm 0.0$
acceptance-subpopulation,$0.995 \pm 0.008$,$0.997 \pm 0.005$
counterfactual-individualized,$0.997 \pm 0.008$,$0.995 \pm 0.005$
improvement-individualized,$1.0 \pm 0.0$,$1.0 \pm 0.0$
improvement-subpopulation,$0.862 \pm 0.085$,$0.852 \pm 0.049$


In [39]:
create_genetic_pivot(refit_pivot_genetic)

,nsga2,nsga3
method,,
acceptance-individualized,$0.863 \pm 0.059$,$0.922 \pm 0.015$
acceptance-subpopulation,$0.869 \pm 0.058$,$0.924 \pm 0.019$
counterfactual-individualized,$0.855 \pm 0.065$,$0.92 \pm 0.026$
improvement-individualized,$1.0 \pm 0.0$,$1.0 \pm 0.0$
improvement-subpopulation,$0.861 \pm 0.087$,$0.855 \pm 0.048$
